In [5]:
! pip install unsloth transformers datasets xformers huggingface_hub torch torchvision torchaudio bitsandbytes peft trl tiktoken wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 213.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 160.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 238.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.8/340.8 kB 147.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 40.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
from huggingface_hub import notebook_login
notebook_login()

# WandB init()
init(): https://docs.wandb.ai/ref/python/init/

In [6]:
import wandb

wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: laravelshubham (laravelshubham-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
run = wandb.init(
    project='Fine-tuning Gemma-3-4B on FinQA Reasoning Dataset', 
    job_type="training", 
    anonymous="allow"
)


In [8]:
from unsloth import FastModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothCPOTrainer: No module named 'UnslothCPOTrainer'. Using tempfile instead!


In [9]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 1. Max memory: 47.5 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [10]:
model

Gemma3ForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(4096, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
            

## Prompt

In [12]:
prompt = """
There is a Question provided below. It has an instruction that describes a task, paired with the input that provides further context to the task.
Write a response that appropriately completes the request.

Before providing the response, think about the question carefully and then create a step-by-step chain of thoughts to ensure that the response is logical and accurate.

### Question:
{}

### Response:
<think>
{}

""".strip()

Dataset: https://huggingface.co/datasets/TheFinAI/Fino1_Reasoning_Path_FinQA/viewer?views%5B%5D=train&sql=--+The+SQL+console+is+powered+by+DuckDB+WASM+and+runs+entirely+in+the+browser.%0A--+Get+started+by+typing+a+query+or+selecting+a+view+from+the+options+below.%0ASELECT+*+FROM+train+LIMIT+1%3B&sql_row=0

In [13]:
question = """
Please answer the given financial question based on the context.
Context: amortization expense , which is included in selling , general and administrative expenses , was $ 13.0 million , $ 13.9 million and $ 8.5 million for the years ended december 31 , 2016 , 2015 and 2014 , respectively . the following is the estimated amortization expense for the company 2019s intangible assets as of december 31 , 2016 : ( in thousands ) .
|2017|$ 10509|
|2018|9346|
|2019|9240|
|2020|7201|
|2021|5318|
|2022 and thereafter|16756|
|amortization expense of intangible assets|$ 58370|
at december 31 , 2016 , 2015 and 2014 , the company determined that its goodwill and indefinite- lived intangible assets were not impaired . 5 . credit facility and other long term debt credit facility the company is party to a credit agreement that provides revolving commitments for up to $ 1.25 billion of borrowings , as well as term loan commitments , in each case maturing in january 2021 . as of december 31 , 2016 there was no outstanding balance under the revolving credit facility and $ 186.3 million of term loan borrowings remained outstanding . at the company 2019s request and the lender 2019s consent , revolving and or term loan borrowings may be increased by up to $ 300.0 million in aggregate , subject to certain conditions as set forth in the credit agreement , as amended . incremental borrowings are uncommitted and the availability thereof , will depend on market conditions at the time the company seeks to incur such borrowings . the borrowings under the revolving credit facility have maturities of less than one year . up to $ 50.0 million of the facility may be used for the issuance of letters of credit . there were $ 2.6 million of letters of credit outstanding as of december 31 , 2016 . the credit agreement contains negative covenants that , subject to significant exceptions , limit the ability of the company and its subsidiaries to , among other things , incur additional indebtedness , make restricted payments , pledge their assets as security , make investments , loans , advances , guarantees and acquisitions , undergo fundamental changes and enter into transactions with affiliates . the company is also required to maintain a ratio of consolidated ebitda , as defined in the credit agreement , to consolidated interest expense of not less than 3.50 to 1.00 and is not permitted to allow the ratio of consolidated total indebtedness to consolidated ebitda to be greater than 3.25 to 1.00 ( 201cconsolidated leverage ratio 201d ) . as of december 31 , 2016 , the company was in compliance with these ratios . in addition , the credit agreement contains events of default that are customary for a facility of this nature , and includes a cross default provision whereby an event of default under other material indebtedness , as defined in the credit agreement , will be considered an event of default under the credit agreement . borrowings under the credit agreement bear interest at a rate per annum equal to , at the company 2019s option , either ( a ) an alternate base rate , or ( b ) a rate based on the rates applicable for deposits in the interbank market for u.s . dollars or the applicable currency in which the loans are made ( 201cadjusted libor 201d ) , plus in each case an applicable margin . the applicable margin for loans will .
Question: what portion of the estimated amortization expense will be recognized in 2017?
Answer:
""".strip()

## Enable Inference

In [16]:
FastModel.for_inference(model) # Enable for inference!

Gemma3ForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(4096, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
            

### Format the prompt

This is a call to Python’s .format() string method, used to substitute values into a predefined template string.

In this case, you have a prompt template (likely a string) with placeholders — like:
```
prompt_style = "Q: {}\nA: {}"
```
Then you call:
```
prompt_style.format(question, "")
```
This will substitute:
```
The first {} with question

The second {} with "" (an empty string — maybe for the answer)
```


In [23]:
prompt_format = prompt.format(question, "")

In [22]:
prompt_format

'There is a Question provided below. It has an instruction that describes a task, paired with the input that provides further context to the task.\nWrite a response that appropriately completes the request.\n\nBefore providing the response, think about the question carefully and then create a step-by-step chain of thoughts to ensure that the response is logical and accurate.\n\n### Question:\nPlease answer the given financial question based on the context.\nContext: amortization expense , which is included in selling , general and administrative expenses , was $ 13.0 million , $ 13.9 million and $ 8.5 million for the years ended december 31 , 2016 , 2015 and 2014 , respectively . the following is the estimated amortization expense for the company 2019s intangible assets as of december 31 , 2016 : ( in thousands ) .\n|2017|$ 10509|\n|2018|9346|\n|2019|9240|\n|2020|7201|\n|2021|5318|\n|2022 and thereafter|16756|\n|amortization expense of intangible assets|$ 58370|\nat december 31 , 2016 

# Tokenizer
https://huggingface.co/docs/tokenizers/training_from_memory

🧠 Conceptual Overview: What is the Unsloth Tokenizer?

The tokenizer in Unsloth is typically:

1. Derived from Hugging Face’s tokenizer (like LLaMATokenizer, AutoTokenizer)

2. Wrapped with extra utilities to support:

    * Instruction tuning (chat format)

    * Vision + Text tokenization

    * Token-efficient formatting

    * Faster .to() support with .cuda() or .to(device)

🛠️ How Unsloth Tokenizer Works Under the Hood

When you call:
```
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
```
This typically:
1. Tokenizes your text prompt
2. Pads/truncates according to max_length
3. Converts to PyTorch tensors
4. Moves them to GPU (.to("cuda"))

✨ Example Output
```
{
  'input_ids': tensor([[1, 29871, 318, 1234, 456, 2]]),
  'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0],
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])
}
```
For vision inputs:
```
inputs = tokenizer(image, text, return_tensors="pt")
```
It will return:
```
{
  'pixel_values': tensor(...),          # image embedding
  'input_ids': tensor(...),            # tokenized prompt
  'attention_mask': tensor(...)
}
```
📦 Special Features in Unsloth’s Tokenizer
Feature: Description

1. apply_chat_template(): Converts instruction + response into formatted prompt
2. to(device): Moves all tensors to a device in one line
3. Multimodal support: Accepts image + text (if Vision model is used)
4. Fast tokenization: Tokenizers may be patched for performance (faster than Hugging Face baseline)
5. Instruction formatting: Integrates system, user, assistant roles for chat training

In [24]:
inputs = tokenizer([prompt_format], return_tensors="pt").to("cuda")

In [25]:
inputs

{'input_ids': tensor([[     2,   3810,    563,    496,  19566,   3847,   3426, 236761,   1030,
            815,    614,  14787,    600,  15517,    496,   4209, 236764,  33481,
            607,    506,   2744,    600,   4728,   3342,   4403,    531,    506,
           4209, 236761,    107,   6974,    496,   3072,    600,  37404,  42342,
            506,   2864, 236761,    108,  13286,   6655,    506,   3072, 236764,
           1751,   1003,    506,   2934,  13058,    532,   1299,   2619,    496,
           2918, 236772,   2003, 236772,   9340,   7797,    529,  12018,    531,
           5330,    600,    506,   3072,    563,  23420,    532,  11459, 236761,
            108,  10354,  19566, 236787,    107,   9366,   3890,    506,   2238,
           4867,   2934,   2721,    580,    506,   4403, 236761,    107,   3637,
         236787, 190375,  18766,   1031,    837,    563,   5012,    528,  11010,
           1031,   2870,    532,  15006,  14348,   1031,    691,    609, 236743,
         23677

### Generate

For decoder-only models inputs should be in the format of input_ids. For encoder-decoder models inputs can represent any of input_ids, input_values, input_features, or pixel_values.



In [26]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1024,
    use_cache=True,
)

In [27]:
outputs

tensor([[     2,   3810,    563,  ..., 236819,    107,    106]],
       device='cuda:0')

In [28]:
response = tokenizer.batch_decode(outputs)

In [29]:
response

['<bos>There is a Question provided below. It has an instruction that describes a task, paired with the input that provides further context to the task.\nWrite a response that appropriately completes the request.\n\nBefore providing the response, think about the question carefully and then create a step-by-step chain of thoughts to ensure that the response is logical and accurate.\n\n### Question:\nPlease answer the given financial question based on the context.\nContext: amortization expense , which is included in selling , general and administrative expenses , was $ 13.0 million , $ 13.9 million and $ 8.5 million for the years ended december 31 , 2016 , 2015 and 2014 , respectively . the following is the estimated amortization expense for the company 2019s intangible assets as of december 31 , 2016 : ( in thousands ) .\n|2017|$ 10509|\n|2018|9346|\n|2019|9240|\n|2020|7201|\n|2021|5318|\n|2022 and thereafter|16756|\n|amortization expense of intangible assets|$ 58370|\nat december 31 ,

In [30]:
question = """
Please answer the given financial question based on the context.
Context: incremental contract start-up costs 2014large municipal contract . during 2018 and 2017 , we incurred costs of $ 5.7 million and $ 8.2 million , respectively , related to the implementation of a large municipal contract . these costs did not meet the capitalization criteria prescribed by the new revenue recognition standard . adoption of the tax act . the tax act was enacted on december 22 , 2017 . among other things , the tax act reduced the u.s . federal corporate tax rate from 35% ( 35 % ) to 21% ( 21 % ) . for the year ended december 31 , 2017 , we recorded provisional amounts based on our estimates of the tax act 2019s effect to our deferred taxes , uncertain tax positions , and one-time transition tax . these adjustments reduced our tax provision by $ 463.9 million . during 2018 , we adjusted the provisional amounts recorded as of december 31 , 2017 for the one-time transition tax , deferred taxes and uncertain tax positions . these adjustments increased our tax provision by $ 0.3 million . bridgeton insurance recovery , net . during 2018 , we collected an insurance recovery of $ 40.0 million related to our closed bridgeton landfill in missouri , which we recognized as a reduction of remediation expenses in our cost of operations . in addition , we incurred $ 12.0 million of incremental costs attributable to the bridgeton insurance recovery . recent developments 2019 financial guidance in 2019 , we will continue to focus on managing the controllable aspects of our business by enhancing the quality of our revenue , investing in profitable growth opportunities and reducing costs . our team remains focused on executing our strategy to deliver consistent earnings and free cash flow growth , and improve return on invested capital . we are committed to an efficient capital structure , maintaining our investment grade credit ratings and increasing cash returned to our shareholders . our guidance is based on current economic conditions and does not assume any significant changes in the overall economy in 2019 . specific guidance follows : revenue we expect 2019 revenue to increase by approximately 4.25 to 4.75% ( 4.75 % ) comprised of the following : increase ( decrease ) .
||increase ( decrease )|
|average yield|2.75% ( 2.75 % )|
|volume|0.0 to 0.25|
|energy services|2013|
|fuel recovery fees|0.25|
|recycling processing and commodity sales|0.25 to 0.5|
|acquisitions / divestitures net|1.0|
|total change|4.25 to 4.75% ( 4.75 % )|
changes in price are restricted on approximately 50% ( 50 % ) of our annual service revenue . the majority of these restricted pricing arrangements are tied to fluctuations in a specific index ( primarily a consumer price index ) as defined in the contract . the consumer price index varies from a single historical stated period of time or an average of trailing historical rates over a stated period of time . in addition , the initial effect of pricing resets typically lags 6 to 12 months from the end of the index measurement period to the date the revised pricing goes into effect . as a result , current changes in a specific index may not manifest themselves in our reported pricing for several quarters into the future. .
Question: what is the ratio of the acquisitions / divestitures net to the fuel recovery fees as part of the expected 2019 revenue to increase
Answer:
""".strip()

In [31]:
prompt_format = prompt.format(question, "")

In [32]:
inputs = tokenizer([prompt_format], return_tensors="pt").to("cuda")

In [33]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1024,
    use_cache=True,
)

In [34]:
response = tokenizer.batch_decode(outputs)

In [35]:
response

['<bos>There is a Question provided below. It has an instruction that describes a task, paired with the input that provides further context to the task.\nWrite a response that appropriately completes the request.\n\nBefore providing the response, think about the question carefully and then create a step-by-step chain of thoughts to ensure that the response is logical and accurate.\n\n### Question:\nPlease answer the given financial question based on the context.\nContext: incremental contract start-up costs 2014large municipal contract . during 2018 and 2017 , we incurred costs of $ 5.7 million and $ 8.2 million , respectively , related to the implementation of a large municipal contract . these costs did not meet the capitalization criteria prescribed by the new revenue recognition standard . adoption of the tax act . the tax act was enacted on december 22 , 2017 . among other things , the tax act reduced the u.s . federal corporate tax rate from 35% ( 35 % ) to 21% ( 21 % ) . for the

In [36]:
response[0].split("### Response:")[1]

'\n<think>\nThe question asks for the ratio of "acquisitions / divestitures net" to "fuel recovery fees" in the expected 2019 revenue increase.\n\nFrom the provided context, we have:\n*   "acquisitions / divestitures net" is expected to increase by 1.0.\n*   "fuel recovery fees" is expected to increase by 0.25.\n\nThe ratio is (1.0) / (0.25) = 4.\n</think>\n4.  0\n<end_of_turn>'

In [37]:
def extract_data(text_list):
    extracted_data = []
    for text in text_list:
        # Find the start and end indices
        start_idx = text.find('</think>\n') + len('</think>\n')
        end_idx = text.find('\n<end_of_turn>', start_idx)
        
        # Extract the text between the markers
        if start_idx != -1 and end_idx != -1:
            extracted = text[start_idx:end_idx].strip()
            extracted_data.append(extracted)
        else:
            extracted_data.append(None)
    
    return extracted_data

In [38]:
extract_data(response)

['4.  0']

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.
